<a href="https://colab.research.google.com/github/giu176/NetworkSecurity-ZKP/blob/main/ZKP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ZERO KNOWLEDGE PROOF


Project for Network Security and Cryptography
by Giulio Bosisio

---

The aim of the simulation is to perform a ZK-SNARK (Zero Knowledge - Succinct Non-Interactive Argument of Knowledge) computation of a python script. With a SNARK proof (a textual file) a Verifier can perform the verification whenever he wants without gaining any information about the problem (the script), the Verifier knows only the output relative to a specific input of the program (Zero Knowledge). Succint means that the computation produce a proof that is both small in size and fast to verify. This proof is also constructed without interacting with the Verifier which makes the computation Non-Interactive, this is an advantage because an Interactive protocol would have required the Verifier to challenge the Prover in real time. 

Properties of Zero Knowledge protocols:
*   Completeness: If the Prover is honest, then it will eventually convince the Verifier.
*   Soundness: The Prover can only convince the Verifier if the statement is true.
*   Zero-knowledge(ness): The Verifier learns no information beyond the fact that the statement is true.



### Example of an Iterative ZK protocol: Standard Schnorr protocol


Schnorr identification protocol is used by a Verifier (V) to prove that a Prover (P) knows the private key x based on the fact that P has previously published a public key y generated using x.

Let G be a cyclic group of prime-order q  generated by g and p a prime number.

The Prover generate the public key y as:
\begin{equation}
y=g^x \mod p
\end{equation}
With:
\begin{equation}
x \in Z_q
\end{equation}

Schnorr protocol:

1.  The Prover picks a random number k (k must not be reused in future iterations of the protocol):
\begin{equation}
k \in Z_q
\end{equation}
then calculate:
\begin{equation}
r=g^k \mod p
\end{equation}
The Prover sends r to the Verifier.

2.   The Verifier sends a random challenge c to the Prover:
\begin{equation}
c \in Z_q
\end{equation}

3.   The Prover comupte s as:
\begin{equation}
s=xc+k \mod p
\end{equation}
Where x is the secret key. The Prover send s  to the Verifier.

4.   The Verifier has y (public key), r, c and s so he can compute:
\begin{equation}
g^s=y^cr
\end{equation}
Which is equivalent to:
\begin{equation}
g^{xc+k}=(g^x)^cg^k
\\
g^{xc+k}=g^{xc+k}
\end{equation}
Last step is possible only if the Prover knows x otherwise he would have provided a wrong value of s to the Verifier.


Properties of Schnorr Protocol: [INCOMPLETE]

*   Completeness: If the Prover is honest the verification process (last step) is always true. This means that the protocol is complete.
*   Soundness: Prove with EXTRACTOR
*   Zero-Knowledgeness: Prove with SIMULATOR

The standard Schnorr protocol is Honest Verifier Zero Knowledge (HVZK) which means the security really only holds in the case that a Verifier is honest. Usually the two parts involved in a Zero Knowledge Proof don't trust each others. To generate a proof against a dishonest verifier the ZKP protocol must be constructed in a Non-Interactive way (like SNARK), this involves the supervision of a Trusted Party.

### Initialization


---

Installing pysnark and creating folders:

In [1]:
!mkdir TrustedParty && mkdir Prover && mkdir Verifier
!pip3 install git+https://github.com/meilof/pysnark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/meilof/pysnark to /tmp/pip-req-build-r812wyhj
  Running command git clone -q https://github.com/meilof/pysnark /tmp/pip-req-build-r812wyhj
  Created wheel for PySNARK: filename=PySNARK-0.3.1-py3-none-any.whl size=116886 sha256=83d0544bfe7939b8803c5c4bb2333646259c4e40b41748ef973a4a6e62819f34
  Stored in directory: /tmp/pip-ephem-wheel-cache-gnqewnya/wheels/ee/ac/e0/e83d7fb143db3e217346b033cccb8100fecdac64bc9d9237ab
Successfully built PySNARK


Default backend for pysnark:
```
!pip3 install python-libsnark
```
libsnark backend doesn't work on Google Colab, other two backends can be used.

### Installing snarkjs backend


---



Updating [npm](https://www.npmjs.com/) and installing [snarkjs](https://github.com/iden3/snarkjs) module along with [r1cs file parser](https://github.com/iden3/r1csfile):

In [ ]:
!npm install -g npm
!npm install snarkjs -g
!npm install https://github.com/iden3/r1csfile.git -g
!npm install r1csfile

/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
+ npm@8.18.0
added 67 packages from 21 contributors, removed 296 packages and updated 138 packages in 7.9s


### Installing qaptools backend


---


Downloading dependencies and building [qaptools](https://github.com/Charterhouse/qaptools):

In [3]:
!sudo apt install build-essential cmake git libgmp3-dev libprocps-dev python3-markdown libboost-program-options-dev libssl-dev python3 pkg-config
!git clone --recursive https://github.com/Charterhouse/qaptools.git
!cd qaptools/ && mkdir build && cd build/ && cmake .. -DCURVE=ALT_BN128 -DUSE_PT_COMPRESSION=OFF -DWITH_PROCPS=OFF -DBINARY_OUTPUT=OFF && make && sudo make install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libgmp3-dev is already the newest version (2:6.1.2+dfsg-2).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python3-markdown is already the newest version (2.6.9-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
git is already the newest version (1:2.17.1-1ubuntu0.12).
libprocps-dev is already the newest version (2:3.3.12-3ubuntu1.2).
libssl-dev is already the newest version (1.1.1-1ubuntu2.1~18.04.20).
python3 is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
fatal: destination path 'qaptools' already exists and is not an empty 

## Trusted Party


---



The example script solves a quadratic equation (as seen in this [article](https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649))


\begin{equation}
x^3+x+5
\end{equation}
 and output the result.

 example.py:


```
import sys

from pysnark.runtime import snark, PrivVal

@snark
def equation(x):
    return x*x*x+x+5

print("output:", equation(int(sys.argv[1])))
```



Downloading the script:

In [4]:
!cd TrustedParty && wget https://raw.githubusercontent.com/giu176/NetworkSecurity-ZKP/main/example.py

--2022-08-26 13:56:57--  https://raw.githubusercontent.com/giu176/NetworkSecurity-ZKP/main/example.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147 [text/plain]
Saving to: ‘example.py’

example.py          100%[===================>]     147  --.-KB/s    in 0s      

2022-08-26 13:56:57 (5.95 MB/s) - ‘example.py’ saved [147/147]



The Trusted Party runs the the script with an input of its choiche. (input x=3 to follow the example in the [article](https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649)).

At this point pysnark execute a runtime script that generates a circuit file and a witness. The circuit is a conversion of the complex mathematical statements inside the python script into simple statements (y = x or y = x [op] z) where [op] is an operator: "+" "-" "*" or "/". 

The quadratic function in the example will be:
\begin{equation}
k=input\times input
\\
y=k\times input
\\
z=y+input
\\
output=z+5
\end{equation}

This is called circuit because the simple statements can be used to build a logic gates circuit using AND gates and OR gates that performs the mathematical operation.

Each logic gate in the circuit is now converted using the standard format R1CS (Rank-1 Constraint System) into a triple of vectors (A,B,C) such as:
\begin{equation}
S\cdot A\times S\cdot B - S\cdot C=0
\end{equation}

Where S is the witness: a vector containing 1, the input, the output and all the solution of the simple statements. 

In our example:

\begin{equation}
S=[1,input,output,k,y,z]
\\
S=[1,3,35,9,27,30]
\end{equation}

Snarkjs uses [circom](https://github.com/iden3/circom) to construct circuit.r1cs which contains every A,B and C vectors for every logic gate and the witness.wtns file that contains S.

Those file can be seen if the snarkjs backend is used otherwise qaptools backend will recieve the circuit directly from the runtime script without producing any file.

### Using snarkjs backend

Setting backend to snarkjs and running the script:

In [ ]:
!cd TrustedParty && PYSNARK_BACKEND=snarkjs python3 example.py 3

output: 35
snarkjs witness.wtns and circuit.r1cs written; see readme


To visualize the [content of circuit.r1cs](https://github.com/iden3/r1csfile/blob/master/doc/r1cs_bin_format.md) execute the print_circuit script (the console log function **doesn't stop by itself**, use the STOP button to exit the console after a few seconds):

In [ ]:
!cd TrustedParty && wget https://raw.githubusercontent.com/giu176/NetworkSecurity-ZKP/main/print_circuit.js
!cd TrustedParty && node print_circuit.js

--2022-08-26 12:54:14--  https://raw.githubusercontent.com/giu176/NetworkSecurity-ZKP/main/print_circuit.js
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98 [text/plain]
Saving to: ‘print_circuit.js’

print_circuit.js    100%[===================>]      98  --.-KB/s    in 0s      

2022-08-26 12:54:15 (4.90 MB/s) - ‘print_circuit.js’ saved [98/98]

{
  n8: 32,
  prime: 21888242871839275222246405745257275088548364400416034343698204186575808495617n,
  curve: {
    q: 21888242871839275222246405745257275088696311157297823662689037894645226208583n,
    r: 21888242871839275222246405745257275088548364400416034343698204186575808495617n,
    name: 'bn128',
    tm: ThreadManager {
      actionQueue: [],
      oldPFree: 0,
      memory: Memory [WebAssembly.Memory] {},

powersoftau  [INCOMPLETE]

In [ ]:
!cd TrustedParty && snarkjs powersoftau new bn128 12 pot.ptau -v

[DEBUG] snarkJS: Calculating First Challenge Hash
[DEBUG] snarkJS: Calculate Initial Hash: tauG1
[DEBUG] snarkJS: Calculate Initial Hash: tauG2
[DEBUG] snarkJS: Calculate Initial Hash: alphaTauG1
[DEBUG] snarkJS: Calculate Initial Hash: betaTauG1
[DEBUG] snarkJS: Blank Contribution Hash:
		786a02f7 42015903 c6c6fd85 2552d272
		912f4740 e1584761 8a86e217 f71f5419
		d25e1031 afee5853 13896444 934eb04b
		903a685b 1448b755 d56f701a fe9be2ce
[INFO]  snarkJS: First Contribution Hash:
		9e63a5f6 2b96538d aaed2372 481920d1
		a40b9195 9ea38ef9 f5f6a303 3b886516
		0710d067 c09d0961 5f928ea5 17bcdf49
		ad75abd2 c8340b40 0e3b18e9 68b4ffef


In [ ]:
!cd TrustedParty && snarkjs powersoftau prepare phase2 pot.ptau pott.ptau -v

[DEBUG] snarkJS: Starting section: tauG1
[DEBUG] snarkJS: tauG1: fft 0 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 0 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 4 mix start: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix start: 1/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 1/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 0/2
[DEBUG] snarkJS: tauG1: fft  4  join: 4/4
[DEBUG] snarkJS: tauG1: fft 4 join  4/4  1/1 0/1
[DEBUG] snarkJS: tauG1: fft 5 mix start: 0/2
[DEBUG] snarkJS: tauG1: fft 5 mix start: 1/2
[DEBUG] snarkJS: tauG1: fft 5 mix end: 1/2
[DEBUG] snarkJS: tauG1: fft 5 mix end: 0/2
[DEBUG] snarkJS: tauG1: fft  5  join: 5/5
[DEBUG] snarkJS: tauG1: fft 5 join  5/5  1/1 0/1
[DEBUG] snarkJS: tauG1: fft 6 mix start: 0/2
[DEBUG] snarkJS: tauG1: fft 

In [ ]:
!cd TrustedParty && snarkjs zkey new circuit.r1cs pott.ptau circuit.zkey

[INFO]  snarkJS: Reading r1cs
[INFO]  snarkJS: Reading tauG1
[INFO]  snarkJS: Reading tauG2
[INFO]  snarkJS: Reading alphatauG1
[INFO]  snarkJS: Reading betatauG1
[INFO]  snarkJS: Circuit hash: 
		125a6161 740573a4 275f05a8 c348813c
		9a23b603 7b53bd5d 9e5bc25c c64ccba6
		2a2e338a a461e599 b2524bd1 4263d7fb
		5cfeda36 489f3652 d4fa036f f345ce14


In [ ]:
!cd TrustedParty && snarkjs zkey export verificationkey circuit.zkey verification_key.json

Distributing files to the Prover:

In [ ]:
!cp ./TrustedParty/circuit.zkey ./Prover/circuit.zkey
!cp ./TrustedParty/witness.wtns ./Prover/witness.wtns

Distributing files to the Verifier:

In [ ]:
!cp ./TrustedParty/verification_key.json ./Verifier/verification_key.json

### Using qaptools backend


qap theory  [INCOMPLETE]

Running the script (qaptools backend is called automatically):

In [5]:
!cd TrustedParty && python example.py 3

*** Error loading backend pysnark.libsnark.backend: No module named 'libsnark'
*** Error loading backend pysnark.libsnark.backendgg: No module named 'libsnark'
output: 35
*** qaptools subroutines:
***    id: main function: main digest: 314adf18c4 #constraints: 5 *
*** generating master key material
*** new signature for function main, rebuilding keys
*** verification succeeded
***  prover keys/eqs:  pysnark_masterek pysnark_ek_main pysnark_eqs_main pysnark_schedule
***  prover data:      
***  verifier keys:    pysnark_masterpk pysnark_vk_main pysnark_schedule
***  verifier data:     pysnark_proof pysnark_values
***  verifier cmd:     qapver pysnark_masterpk pysnark_schedule pysnark_proof pysnark_values


Using commitments: [INCOMPLETE]

In [6]:
!cd TrustedParty && python -m pysnark.qaptools.runqapinput test 1 2 3

*** enlarging master key material
*** building block commitment pysnark_comm_test from wires pysnark_wires_test


Distributing files to the Prover:


*   example.py: python script.
*   pysnark_schedule: schedule of functions called in the computation.
*   pysnark_masterek: master evaluation key.
*   pysnark_ek_main: zk-SNARK evaluation key for the main function of the computation.
*   pysnark_eqs_main: equations for the main function of the computation.
*   pysnark_masterpk: master public key.


In [9]:
!cp ./TrustedParty/example.py ./Prover/example.py
!cp ./TrustedParty/pysnark_schedule ./Prover/pysnark_schedule
!cp ./TrustedParty/pysnark_masterek ./Prover/pysnark_masterek
!cp ./TrustedParty/pysnark_ek_main ./Prover/pysnark_ek_main
!cp ./TrustedParty/pysnark_eqs_main ./Prover/pysnark_eqs_main
!cp ./TrustedParty/pysnark_masterpk ./Prover/pysnark_masterpk

Distributing files to the Verifier:
*   pysnark_schedule: schedule of functions called in the computation.
*   pysnark_masterpk: master public key.
*   pysnark_vk_main: verificaiton key for the main function.

In [10]:
!cp ./TrustedParty/pysnark_schedule ./Verifier//pysnark_schedule
!cp ./TrustedParty/pysnark_masterpk ./Verifier/pysnark_masterpk
!cp ./TrustedParty/pysnark_vk_main ./Verifier/pysnark_vk_main

## Prover


---



### Using snarkjs backend

Running the proof:

In [ ]:
!cd Prover && snarkjs groth16 prove circuit.zkey witness.wtns proof.json public.json

Distributing files to the Verifier:

In [ ]:
!cp ./Prover/public.json ./Verifier/public.json
!cp ./Prover/proof.json ./Verifier/proof.json

### Using qaptools backend

In [11]:
!cd Prover && python example.py 3

*** Error loading backend pysnark.libsnark.backend: No module named 'libsnark'
*** Error loading backend pysnark.libsnark.backendgg: No module named 'libsnark'
output: 35
*** qaptools subroutines:
***    id: main function: main digest: 314adf18c4 #constraints: 5 *
*** verification keys missing, skipping verification
***  prover keys/eqs:  pysnark_masterek pysnark_ek_main pysnark_eqs_main pysnark_schedule
***  prover data:      
***  verifier keys:    pysnark_masterpk pysnark_vk_main pysnark_schedule
***  verifier data:     pysnark_proof pysnark_values
***  verifier cmd:     qapver pysnark_masterpk pysnark_schedule pysnark_proof pysnark_values


Distributing files to the Verifier:
*   pysnark_proof: proof that the particular computation was performed correctly.
*   pysnark_values: input/output values of the computation.

In [12]:
!cp ./Prover/pysnark_proof ./Verifier/pysnark_proof
!cp ./Prover/pysnark_values ./Verifier/pysnark_values

## Verifier


---



### Using snarkjs backend

In [ ]:
!cd Verifier && snarkjs groth16 verify verification_key.json public.json proof.json

[INFO]  snarkJS: OK!


### Using qaptools backend

In [13]:
!cd Verifier && python -m pysnark.qaptools.runqapver

## References


---



*    [Zero Knowledge Proofs: An illustrated primer (Part 1)](https://blog.cryptographyengineering.com/2014/11/27/zero-knowledge-proofs-illustrated-primer/)
*   [Zero Knowledge Proofs: An illustrated primer (Part 2)](https://blog.cryptographyengineering.com/2017/01/21/zero-knowledge-proofs-an-illustrated-primer-part-2/)
*   [The missing explanation of ZK-SNARKs: Part 1](https://medium.com/zeroknowledge/the-missing-explanation-of-zk-snarks-part-1-d9703cb80b91)
*   [zkSnarks: From circuits to verification](https://asecuritysite.com/zero/zksnark03)
*   [R1CS to QAP (quadratic arithmetic program) with zkSNARKs in Go](https://asecuritysite.com/zero/go_qap)
*   [Quadratic Arithmetic Programs: from Zero to Hero](https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649)
*   [Explaining Quadratic Arithmetic Programs](https://xord.com/research/explaining-quadratic-arithmetic-programs/)